## Desafio 2


Construa um detector de bocejos que por meio da identificação de regiões da face calcule a área da boca e indique quando há ou não um bocejo.
Crie um contador de bocejos e que as informações sejam exibidas em tempo real utilizando uma webcam.

In [6]:
import cv2
from matplotlib import pyplot as plt
import numpy as np

In [7]:
import dlib
import numpy 

predictor_68_path = "../modelos/shape_predictor_68_face_landmarks.dat"

predictor = dlib.shape_predictor(predictor_68_path)
detector = dlib.get_frontal_face_detector()

In [8]:
FACE_POINTS = list(range(17, 68))
MOUTH_POINTS = list(range(48, 61))
RIGHT_BROW_POINTS = list(range(17, 22))
LEFT_BROW_POINTS = list(range(22, 27))
RIGHT_EYE_POINTS = list(range(36, 42))
LEFT_EYE_POINTS = list(range(42, 48))
NOSE_POINTS = list(range(27, 35))
JAW_POINTS = list(range(0, 17))

In [9]:
def annotate_landmarks_convex_hull_image(im):
    im = im.copy()
    rects = detector(im, 1)
    
    if len(rects) == 0:
        return im, 0
    
    landmarks_list = []
    
    for rect in rects:
        landmarks = numpy.matrix([[p.x, p.y] for p in predictor(im, rect).parts()])

        for k, d in enumerate(rects):
            cv2.rectangle(im, (d.left(), d.top()), (d.right(), d.bottom()), (255, 0, 255), 2)

            points = cv2.convexHull(landmarks[NOSE_POINTS])
            cv2.drawContours(im, [points], 0, (0, 255, 0), 1)

            points = cv2.convexHull(landmarks[MOUTH_POINTS])
            cv2.drawContours(im, [points], 0, (0, 255, 0), 1)

            area_boca = cv2.contourArea(points)   
            cv2.putText(im, "Boca "+str(area_boca), (d.left(),d.top()-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 1)
            
            points = cv2.convexHull(landmarks[RIGHT_BROW_POINTS])
            cv2.drawContours(im, [points], 0, (0, 255, 0), 1)

            points = cv2.convexHull(landmarks[LEFT_BROW_POINTS])
            cv2.drawContours(im, [points], 0, (0, 255, 0), 1)

            points = cv2.convexHull(landmarks[RIGHT_EYE_POINTS])
            cv2.drawContours(im, [points], 0, (0, 255, 0), 1)

            points = cv2.convexHull(landmarks[LEFT_EYE_POINTS])
            cv2.drawContours(im, [points], 0, (0, 255, 0), 1)

    return im, area_boca

In [10]:
import time

cam_capture = cv2.VideoCapture("video/bliss.avi")

while True:
    ret, image_frame = cam_capture.read()
    
    if ret:
        matches = detectar_embalagem_orb(image_frame)
    
        if matches > 9:
            cv2.putText(image_frame, "Identificado " + str(matches), (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 6)
    
        cv2.imshow("Sketcher ROI", image_frame)

        if cv2.waitKey(1) == 13:
            break
    else:
        break
    time.sleep(0.01)
        
cam_capture.release()
cv2.destroyAllWindows()

In [13]:
#cam_capture.release()
    
cam_capture = cv2.VideoCapture(0)
cv2.destroyAllWindows()

qtde_bocejos = 0

bocejo = False

while True:
    ret, image_frame = cam_capture.read()
    prev_bocejo = bocejo 
    
    if ret:
        image_frame, area_boca = annotate_landmarks_convex_hull_image(image_frame)
        
        print(area_boca)
        
        if area_boca > 2100:
            bocejo = True
        else:
            bocejo = False

        if prev_bocejo == True and bocejo == False:
            qtde_bocejos += 1

        cv2.putText(image_frame, "Bocejo "+str(qtde_bocejos), (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 1)

        cv2.imshow("Detector de Bocejos", image_frame)

        if cv2.waitKey(1) == 13:
            break
    else:
        break
        
cam_capture.release()
cv2.destroyAllWindows()

836.5
823.5
858.5
838.5
855.5
852.5
871.0
793.0
837.5
819.0
868.5
812.0
835.0
834.5
834.0
989.5
1689.0
1668.0
1582.5
1486.0
1460.5
1426.0
1381.5
1349.5
1435.5
1749.0
1425.0
1338.5
1432.5
1391.0
921.0
882.5
853.5
849.5
839.5
831.0
800.5
1556.0
1582.5
1649.0
1645.5
1680.0
1762.5
1614.0
1783.5
1778.5
1578.5
2003.0
1910.5
2104.0
2045.0
2214.5
2233.0
2004.0
864.0
776.5
786.0
769.0
826.0
766.5
668.5
644.5
654.0
718.5
659.0
649.5
789.5
928.5
869.5
884.5
900.5
842.5
870.0
838.5
823.5
827.0
873.0
846.0
803.5
826.0
844.5
818.5
872.0
843.0
